In [6]:
import numpy as np
import tensorflow as tf
import os
#from riptide.models.cifar_resnet import cifar_resnet20_v1
#from riptide.binary.q_cifar_resnet import cifar_resnet20_v1
from riptide.models.resnetv1b import resnet18_v1b
from riptide.binary.HWGQ_funcs import Quantize, HWGQuantize, load_clusters, load_bits
from riptide.binary.HWGQ_layers import Config
from slim.preprocessing.inception_preprocessing import preprocess_image
from riptide.utils.datasets import get_imagenet_dataset
from functools import partial

In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [9]:
batch_size=64

train_preprocess = partial(preprocess_image, height=224, width=224, is_training=True)
def train_input_fn():
     return get_imagenet_dataset(batch_size, preprocess=train_preprocess, is_training=True)

val_preprocess = partial(preprocess_image, height=224, width=224, is_training=False)

def eval_input_fn():
    return get_imagenet_dataset(batch_size, preprocess=val_preprocess, is_training=False)

In [12]:
model = resnet18_v1b()

In [17]:
test_data = tf.random_normal(shape=(1, 224, 224, 3))
output= model(test_data)

NotImplementedError: 

In [15]:
model.layer1.summary()

ValueError: This model has never been called, thus its weights have not yet been created, so no summary can be displayed. Build the model first (e.g. by calling it on some data).

In [7]:
def cnn_model_fn(features, labels, mode):
    """Model function for CNN."""
    #clusters = tf.constant(load_clusters(2))
    #actQ = HWGQuantize
    #weightQ = Quantize
    #config = Config(actQ=actQ, weightQ=weightQ, clusters=clusters)
    #with config:
    #    model = ()
    model = resnet18_v1b()
    logits = model(features)
    predictions = {
        # Generate predictions (for PREDICT and EVAL mode)
        "classes": tf.argmax(input=logits, axis=1),
        # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
        # `logging_hook`.
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        #optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        lr = tf.train.exponential_decay(0.1, tf.train.get_global_step(), 20000, 0.1)
        optimizer = tf.train.MomentumOptimizer(learning_rate = lr, momentum=0.9)
        #optimizer = tf.train.AdamOptimizer(learning_rate = lr)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [10]:
# Create the Estimator
strategy = tf.contrib.distribute.MirroredStrategy(num_gpus=3)
run_config = tf.estimator.RunConfig(train_distribute=strategy)
mnist_classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fn, model_dir="/tmp/resnet_v1")
train_spec = tf.estimator.TrainSpec(input_fn=train_input_fn, max_steps=None)
eval_spec = tf.estimator.EvalSpec(input_fn=eval_input_fn)

INFO:tensorflow:Initializing RunConfig with distribution strategies.
INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_global_id_in_cluster': 0, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f69e985bb50>, '_model_dir': '/tmp/resnet_v1', '_protocol': None, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_device_fn': None, '_experimental_distribute': None, '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_evaluation_master': '', '_eval_distribute': None, '_train_distribute': None, '_master': ''}


In [11]:
tf.estimator.train_and_evaluate(mnist_classifier, train_spec, eval_spec)

INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps None or save_checkpoints_secs 600.
INFO:tensorflow:Calling model_fn.


NotImplementedError: 

In [ ]:
eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)